In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

#Частина 1

In [28]:
# Завантаження та підготовка даних
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Нормалізація зображень
train_images = train_images / 255.0
test_images = test_images / 255.0

# Зміна розміру зображень для згорткової мережі
train_images = train_images.reshape((-1, 28, 28, 1))
test_images = test_images.reshape((-1, 28, 28, 1))

# One-hot кодування міток
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

# Створення моделі
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(10, activation='softmax')
])

# Компіляція моделі
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',Precision(), Recall(), AUC()])

# Навчання моделі
history = model.fit(train_images, train_labels_one_hot, epochs=10, batch_size=64, validation_split=0.2)



Epoch 1/10
750/750 [==============================] - 44s 58ms/step - loss: 0.7166 - accuracy: 0.7742 - precision_8: 0.8569 - recall_8: 0.7021 - auc_8: 0.9789 - val_loss: 0.5145 - val_accuracy: 0.8398 - val_precision_8: 0.8757 - val_recall_8: 0.8040 - val_auc_8: 0.9883
Epoch 2/10
750/750 [==============================] - 41s 55ms/step - loss: 0.4672 - accuracy: 0.8538 - precision_8: 0.8852 - recall_8: 0.8231 - auc_8: 0.9900 - val_loss: 0.3957 - val_accuracy: 0.8748 - val_precision_8: 0.9012 - val_recall_8: 0.8538 - val_auc_8: 0.9927
Epoch 3/10
750/750 [==============================] - 41s 55ms/step - loss: 0.3996 - accuracy: 0.8735 - precision_8: 0.8969 - recall_8: 0.8499 - auc_8: 0.9922 - val_loss: 0.3670 - val_accuracy: 0.8819 - val_precision_8: 0.9043 - val_recall_8: 0.8632 - val_auc_8: 0.9934
Epoch 4/10
750/750 [==============================] - 41s 55ms/step - loss: 0.3678 - accuracy: 0.8828 - precision_8: 0.9050 - recall_8: 0.8614 - auc_8: 0.9932 - val_loss: 0.3425 - val_accura

In [29]:
test_loss, test_accuracy, test_precision, test_recall, test_auc = model.evaluate(test_images[-1500:], test_labels_one_hot[-1500:])

print(f"Accuracy: {test_accuracy * 100:.2f}%")
print(f"Precision: {test_precision:.2f}")
print(f"Recall: {test_recall:.2f}")
print(f"AUC: {test_auc:.2f}")

47/47 [==============================] - 0s 8ms/step - loss: 0.2908 - accuracy: 0.9120 - precision_8: 0.9273 - recall_8: 0.8933 - auc_8: 0.9960
Accuracy: 91.20%
Precision: 0.93
Recall: 0.89
AUC: 1.00


# Print the metrics

In [ ]:
test_labels_one_hot = to_categorical(test_labels)
test_loss, test_accuracy, test_precision, test_recall, test_auc = model.evaluate(test_images[-1500:], test_labels_one_hot[-1500:])

print(f"Accuracy: {test_accuracy * 100:.2f}%")
print(f"Precision: {test_precision:.2f}")
print(f"Recall: {test_recall:.2f}")
print(f"AUC: {test_auc:.2f}")

47/47 [==============================] - 0s 8ms/step - loss: 0.4261 - accuracy: 0.8387 - precision_30: 0.8767 - recall_30: 0.8107 - auc_30: 0.9885
Accuracy: 83.87%
Precision: 0.88
Recall: 0.81
AUC: 0.99


#Частина 2

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
def to_rgb(img):
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

In [ ]:
training_images_rgb = [to_rgb(img) for img in training_images]
test_images_rgb = [to_rgb(img) for img in test_images]

In [ ]:
training_images_rgb = np.array(training_images_rgb) / 255.0
test_images_rgb = np.array(test_images_rgb) / 255.0

In [ ]:
training_labels_one_hot = to_categorical(training_labels, num_classes=10)
test_labels_one_hot = to_categorical(test_labels, num_classes=10)

In [ ]:
vgg = VGG16(input_shape=(32, 32, 3), weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(10, activation='softmax')(x)




In [ ]:
model1 = Model(inputs=vgg.input, outputs=prediction)


model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',  # Using categorical_crossentropy since we have one-hot encoded labels
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

In [ ]:

model1.fit(training_images_rgb, training_labels_one_hot, epochs=5, batch_size=32)  # Ensure batch_size aligns with your data

Epoch 1/5
1875/1875 [==============================] - 626s 333ms/step - loss: 0.7094 - accuracy: 0.7710 - precision_1: 0.8802 - recall_1: 0.6309 - auc_1: 0.9750
Epoch 2/5
1875/1875 [==============================] - 619s 330ms/step - loss: 0.4928 - accuracy: 0.8278 - precision_1: 0.8818 - recall_1: 0.7687 - auc_1: 0.9864
Epoch 3/5
1875/1875 [==============================] - 621s 331ms/step - loss: 0.4511 - accuracy: 0.8396 - precision_1: 0.8861 - recall_1: 0.7912 - auc_1: 0.9883
Epoch 4/5
1875/1875 [==============================] - 621s 331ms/step - loss: 0.4284 - accuracy: 0.8476 - precision_1: 0.8896 - recall_1: 0.8043 - auc_1: 0.9893
Epoch 5/5
1875/1875 [==============================] - 620s 331ms/step - loss: 0.4138 - accuracy: 0.8514 - precision_1: 0.8905 - recall_1: 0.8118 - auc_1: 0.9899


In [ ]:
test_loss, test_accuracy, test_precision, test_recall, test_auc = model1.evaluate(test_images_rgb[-1500:], test_labels_one_hot[-1500:])

print(f"Accuracy: {test_accuracy * 100:.2f}%")
print(f"Precision: {test_precision:.2f}")
print(f"Recall: {test_recall:.2f}")
print(f"AUC: {test_auc:.2f}")

47/47 [==============================] - 16s 338ms/step - loss: 0.4656 - accuracy: 0.8333 - precision_1: 0.8681 - recall_1: 0.7940 - auc_1: 0.9865
Accuracy: 83.33%
Precision: 0.87
Recall: 0.79
AUC: 0.99


Подібні загальні показники: Обидві мережі показують порівнянно високі показники точності, точності (precision), чутливості (recall) та площі під кривою (AUC). Це свідчить про те, що обидві моделі досить ефективно класифікують зображення за допомогою даних Fashion MNIST.

Незважаючи на трохи нижчу продуктивність, використання передтренованих мереж, як VGG16, може бути корисним у випадках обмежених обчислювальних ресурсів або коли потрібно швидке розгортання моделі. Вони також можуть бути корисними при роботі з дуже складними наборами даних, де потрібно вже передтреноване розуміння базових зображень.

У підсумку, обидві моделі показали себе досить ефективними, але   CNN злегка перевершила VGG16, що демонструє важливість специфічної настройки архітектури для конкретних даних та завдань.